In [1]:
from os.path import join
import os
import pandas as pd
import sys
base_dir = "D:/Thesis/Cobra/cobra/"
if base_dir not in sys.path:
    sys.path.append(base_dir)
from utilities import utils
import importlib
import numpy as np
importlib.reload(utils)

<module 'utilities.utils' from 'D:/Thesis/Cobra/cobra\\utilities\\utils.py'>

In [2]:
base_dir = "D:/Thesis/Cobra/cobra/"
csv_folder = join(base_dir, 'data', 'tables')
neg_tab_dirs = sorted([f"{csv_folder}/{x}" for x
                       in os.listdir(csv_folder) \
                           if x.startswith('healthy')])

### Keys to add

In [58]:
dst = pd.read_csv(join(csv_folder,'days_since_test.csv'))
dst2 = pd.read_csv(join(csv_folder,'days_since_test2.csv'))

In [63]:
dst_comp = dst2[dst2.SeriesInstanceUID.isin(dst.SeriesInstanceUID.unique())]
dst_comp = dst_comp.sort_values(by=['PatientID', 'SeriesInstanceUID'])
dst = dst.sort_values(by=['PatientID', 'SeriesInstanceUID'])

In [ ]:
def match_compare(df1, df2, match_cols, compare_col):
    df1 = df1.sort_values(by=match_cols)
    df2 = df2.sort_values(by=match_cols)
    comparison_mask = np.where(df1.reset_index(drop=True)[compare_col]==df2.reset_index(drop=True)[compare_col],
                            True, False)
    df1 = df1[~comparison_mask].rename(columns={compare_col:'df1_'+compare_col})
    df2 = df2[~comparison_mask].rename(columns={compare_col:'df2_'+compare_col})
    df_diff = pd.concat(df1, df2, axis=1,)
    return df_diff
    

In [72]:
df_diff = pd.concat([dst,dst_comp]).drop_duplicates(
    subset=['PatientID','SeriesInstanceUID','days_since_test'], keep=False)

In [79]:
dst[dst.PatientID=='00438b59ef2f432c390cf99ac82e1244']

,PatientID,SeriesInstanceUID,days_since_test
2315,00438b59ef2f432c390cf99ac82e1244,40c289ef34d144a0213aeb2d218f50d4,-26
2316,00438b59ef2f432c390cf99ac82e1244,6b1323f178a4b9496cd1d6de43fd1104,-26
2317,00438b59ef2f432c390cf99ac82e1244,b85580fed5983dd1586297db51204c2c,-26
2318,00438b59ef2f432c390cf99ac82e1244,c5d7380d9b85bedeea81c2bcc58962d4,-26


In [73]:
df_diff

,PatientID,SeriesInstanceUID,days_since_test
2315,00438b59ef2f432c390cf99ac82e1244,40c289ef34d144a0213aeb2d218f50d4,-26
2316,00438b59ef2f432c390cf99ac82e1244,6b1323f178a4b9496cd1d6de43fd1104,-26
2317,00438b59ef2f432c390cf99ac82e1244,b85580fed5983dd1586297db51204c2c,-26
2318,00438b59ef2f432c390cf99ac82e1244,c5d7380d9b85bedeea81c2bcc58962d4,-26
0,03a79cb80a081f76dc8b8b358baf775e,315484a58e48568df73d92bd5035a712,-453
...,...,...,...
1036,fee7aed833af0d6785bccc337e933a23,d28716cddf4f064b6de31fede726515f,-168
1037,fee7aed833af0d6785bccc337e933a23,dd22a72a3177b089266b0a3552396374,-168
1038,fee7aed833af0d6785bccc337e933a23,e27af30cacc9a7a59cfa7109699fcccf,-168
1039,fee7aed833af0d6785bccc337e933a23,e2c33d11421231f06644fbfc063a76f9,-168


In [82]:
comparison_column = np.where(dst.reset_index(drop=True).days_since_test==dst_comp.reset_index(drop=True).days_since_test,
    True, False)

In [101]:
comparison_column

array([ True,  True,  True, ...,  True,  True,  True])

In [94]:
dst_comp[~comparison_column]

,PatientID,SeriesInstanceUID,days_since_test
5855,00438b59ef2f432c390cf99ac82e1244,40c289ef34d144a0213aeb2d218f50d4,38
5856,00438b59ef2f432c390cf99ac82e1244,6b1323f178a4b9496cd1d6de43fd1104,38
5858,00438b59ef2f432c390cf99ac82e1244,b85580fed5983dd1586297db51204c2c,38
5860,00438b59ef2f432c390cf99ac82e1244,c5d7380d9b85bedeea81c2bcc58962d4,38
455,03a79cb80a081f76dc8b8b358baf775e,315484a58e48568df73d92bd5035a712,-430
...,...,...,...
1036,fee7aed833af0d6785bccc337e933a23,d28716cddf4f064b6de31fede726515f,-168
1037,fee7aed833af0d6785bccc337e933a23,dd22a72a3177b089266b0a3552396374,-168
1038,fee7aed833af0d6785bccc337e933a23,e27af30cacc9a7a59cfa7109699fcccf,-168
1039,fee7aed833af0d6785bccc337e933a23,e2c33d11421231f06644fbfc063a76f9,-168


In [7]:
df_keys = pd.read_csv(join(csv_folder, 'neg_pos.csv')).keys()
dfc_keys = pd.read_csv(join(csv_folder, 'neg_pos_clean.csv')).keys()
missing_keys = set(dfc_keys).difference(set(df_keys))
print(missing_keys)

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (4,5,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


{'RowSpacing', 'ColumnSpacing', 'NumberOfSlices', 'days_since_test'}


C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Merge all negatives

In [4]:

df_neg_all = pd.read_csv(join(csv_folder, 'neg_all.csv'))

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df_neg_all[df_neg_all.InstanceCreationDate.isnull()]

,SeriesInstanceUID,StudyInstanceUID,PatientID,AngioFlag,AcquisitionMatrix,AcquisitionContrast,AcquisitionDuration,dBdt,EchoTime,EchoTrainLength,...,SequenceName,ScanOptions,SeriesDescription,SoftwareVersions,SliceThickness,StudyPriorityID,PatientPosition.1,SpacingBetweenSlices,SecondEcho,VariableFlipAngleFlag
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315925,745dd7d98c52d643b6fd250fb3c8ca8b,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 250, 0]",NaN,NaN,0.0,333.00,137.0,...,*spcir3d1_274ns,"['IR', 'PFP', 'SAT1']",FLAIR cor,syngo MR B19,1.0,LOW,HFS,NaN,NaN,Y
315928,aa5f07e706901e9868311b62cbb76d38,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 256, 0]",NaN,NaN,0.0,2.92,1.0,...,*tfl3d1_ns,IR,MPR tra,syngo MR B19,1.0,LOW,HFS,NaN,NaN,N
315930,b6b0117a50349a9510472a4c183624a8,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 256, 0]",NaN,NaN,0.0,2.92,1.0,...,*tfl3d1_ns,IR,MPR cor +Gd,syngo MR B19,1.0,LOW,HFS,NaN,NaN,N
315932,d558cb734128ac0c344e017d211c4baa,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 256, 0]",NaN,NaN,0.0,2.92,1.0,...,*tfl3d1_ns,IR,MPR cor,syngo MR B19,1.0,LOW,HFS,NaN,NaN,N


In [3]:
df_neg_list = [pd.read_csv(csv_path, encoding= 'unicode_escape') for csv_path in
               neg_tab_dirs]
df_all_neg_2019 = pd.concat(df_neg_list, axis=0, join="outer")
# In[Write it to a csv]
df_all_neg_2019.to_csv(f"{csv_folder}/neg_all.csv", index=False,
 header=True)

ipykernel_launcher:7: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.


# Merge negatives and positives

In [9]:
df_pos = utils.load_scan_csv(join(csv_folder, 'pos.csv'))

In [12]:
df_all = pd.concat([df_all_neg_2019, df_pos], axis=0, join="outer")
df_all.to_csv(f"{csv_folder}/neg_pos.csv", index=False,
 header=True)

In [13]:
df_all.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [14]:
df_pred = pd.read_csv(f"{csv_folder}/pred_seq.csv")

# Add Sequence Prediction

In [17]:
df_all = df_all.merge(df_pred.drop(labels=['PatientID', 'InstanceCreationDate'], axis=1), on='SeriesInstanceUID', how='inner')

In [20]:
df_directory = pd.read_csv(f"{csv_folder}/series_directories_sif.csv")
df_all_final = df_all.merge(df_directory, on='SeriesInstanceUID', how='inner')
df_all_final.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [21]:
df_all_final = df_all_final.rename(columns={'Directory': 'DirectorySIF'})

In [ ]:
def pixelspacing_convert(x, axis): 
    try:
        return x[axis]
    except:
        None
df_all['xPixelSpacing'] = df_all.PixelSpacing.map(lambda x: pixelspacing_convert(x, 0))
df_all['yPixelSpacing'] = df_all.PixelSpacing.map(lambda x: pixelspacing_convert(x, 1))

## Remember to add Positive column and DateTime

In [22]:
df_all.to_csv(f"{csv_folder}/neg_pos.csv", index=False, header=True)

## Clean

In [26]:
df_clean = df_all[~(df_all.Sequence=='other')]
print(f"after removing other sequences there are {len(df_clean)} volumes")

after removing other sequences there are 207027 volumes


In [27]:
df_cclean = df_clean[~df_clean.InstanceCreationDate.isnull()]
print(f"after removing sequences with no date there are {len(df_cclean)} volumes")

after removing sequences with no date there are 178578 volumes


In [28]:
df_cclean.to_csv(f"{csv_folder}/neg_pos_clean.csv", index=False, header=True)